# FP Yield for Franken Sample

This is the yield that would be estimated using the current franken sample approach, a average fission energy of 4.32 MeV (corresponding to the ETA TN+PFNS spectrum), and linear combination of the ENDF/B-VII.1 FP yield data for thermal and high energy neutrons.

In [1]:
import os
import sys

import numpy as np
from math import sqrt
from collections import defaultdict

sys.path.insert(0,os.path.abspath('../'))
from FP_Utilities import Read_E_Bins, Build_Nagy_Weighted_FPs, Build_Spline_Weighted_FPs, Build_GEF_Weighted_FPs
from FP_Utilities import Read_Fission_Spectrum, writeout, Build_Linear_Weighted_FPs

rootdir=os.path.abspath(os.getcwd())

In [2]:
# Determine fission fraction by isotope
# From ETA Ouput - Not explicitly required for program
tot=1.32237E-05
tot_235=1.32237E-05
wf_235=1.0 

print "235",tot_235/tot


235 1.0


In [3]:
# User Inputs
#---------------------------------------------------------------------------------------#  
fiss235Name = 'Data/E_fiss_235.csv'   # Fission spectra input file
binsName = 'Data/Bins.csv'   # Bin structure input file
fp235Name = 'Data/235_data.xlsx'  # 235 energy dependent data - must contain same isotopes as 238
gef235Dir = '../../Validation/235Watt' # Path to GEF data

f235 = 1.0       # fraction of fissions caused by 235 - f_235+f_238 should equal 1
#---------------------------------------------------------------------------------------#  

In [4]:
### Linear ENDF Based Predictions ###

# Read in the energy bin structure
(lower_bins,upper_bins,bins)=Read_E_Bins(binsName)

# Calculate 235 data
(fiss_235_e,fiss_235)=Read_Fission_Spectrum(fiss235Name)
y, err=Build_Linear_Weighted_FPs(fp235Name,bins,f235,fiss_235)

# Ouput the data
absErr = {}
out = []
for A in y.keys():
    absErr[A]=err[A]*y[A]
    out.append((A,y[A],absErr[A]))

# Output Results
print y, absErr
writeout("FrankenSample_Linear_fy.csv",sorted(out,key=lambda l:l[0]))

{97: 5.722518, 99: 5.8070596999999999, 147: 2.0534232999999995, 133: 6.2810624999999991, 161: 0.0016889610000000001, 140: 5.6814102000000002, 141: 5.4249342999999994, 111: 0.34556687199999997, 113: 0.27645676899999999, 115: 0.34636635799999999, 151: 0.39976679000000004, 153: 0.17230898, 156: 0.028181264000000001, 95: 6.0910972000000001} {97: 1.1987576602146726, 99: 0.62156573487547551, 133: 25.371752228550903, 161: 3.9561780673939123e-07, 140: 0.49448374016947566, 141: 0.85453799716962808, 111: 0.0058004335483897215, 115: 0.0049060573587500012, 113: 0.016572613956559581, 147: 0.074688899839070289, 151: 0.013031357902193688, 153: 0.0032530104236859359, 156: 2.896986889538395e-05, 95: 0.67353061556550264}


In [5]:
### GEF Based Predictions ###

# Read in the energy bin structure
(lower_bins,upper_bins,bins)=Read_E_Bins(binsName)

# Calculate 235 data
y={}
err={}
absErr={}

(y,err)=Build_GEF_Weighted_FPs(bins,f235*fiss_235,gef235Dir)

# Combine data into a single data set
out=[]
for A in y.keys():
    try:
        absErr[A]=y[A]*err[A]
        out.append((A,y[A],err[A],absErr[A]))
    except KeyError as e:
        print "Key error: {0} not found for both isotopes.".format(A)

# Output Results
writeout("FrankenSample_GEF_fy.csv",sorted(out,key=lambda l:l[0]))





47 2 2
